In [4]:
import json

In [5]:
def load_json(file):
    f = open(file, 'r')
    return json.load(f)

In [6]:
def write_json(data):
    encoded_data = json.JSONEncoder(indent=2).encode(data)
    
    f = open('mud_of_babel_data.json', 'w')
    f.write(encoded_data)

In [7]:
def build_basic_game_info():
    return {
        'commandCounter' : 0,
        'gameOver' : False,
        'introText' : '\n\nWelcome to the MUD of Babel!\n',
        'outroText' : 'Thanks For playing!',
        'player' : {
            'currentLocation' : '',
            'inventory' : {},
            'lightSource' : False
        },
        'map': {}
    }

In [13]:
def build_room(key, name, description, exits={}, interactables={}, items={}):
    return {
        key: {
            'firstVisit' : True,
            'displayName' : name,
            'description' : description,
#             'setup' : 'function(){end();}',
            'interactables' : interactables,
            'items' : items,
            'exits' : exits
        }
    }


In [135]:
def build_plaza(subject, code, children):
    _child_count = len(children)
    name = '%s Plaza' % subject
    be_str = 'are' if _child_count != 1 else 'is'
    building_str = 'buildings' if _child_count != 1 else 'building'
    their_str = 'their' if _child_count != 1 else 'its'
    description = 'You are in an expansive plaza surrounded by %s %s conforming %s.\nThe %s %s accessible via %s classification code, visible above the main door:' % (_child_count, building_str, subject, building_str, be_str, their_str)
    exits = {}
    entrances = []
    for key, val in children:
        exit_key = "%s_%s" % (code, val['id'])
        entrances.append('[%s] “%s” Building' % (exit_key, val['subject']))
        exits.update({
            exit_key : {
                'displayName' : exit_key,
                'destination' : exit_key
            }
        })
    description = '%s %s.' % (description, ', '.join(entrances))
    return build_room(key="Plaza", name=name, description=description, exits=exits)

In [136]:
def build_building(subject, code, children):
    _child_count = len(children)
    be_str = 'are' if _child_count != 1 else 'is'
    room_str = 'rooms' if _child_count != 1 else 'room'
    description = 'There %s %s %s throughout this building. Visit them with the appropriate code.' % (be_str, _child_count, room_str)
    exits = {
        code : {
            'displayName' : 'Main Plaza',
            'destination' : 'Plaza'
        }
    }
    entrances = []
    for key, val in children:
        exit_key = "%s_%s" % (code, val['id'])
        entrances.append('[%s] “%s” Room' % (exit_key, val['subject']))
        exits.update({
            exit_key : {
                'displayName' : exit_key,
                'destination' : exit_key
            }
        })
    description = '%s %s.' % (description, ', '.join(entrances))
    return build_room(key=code, name=subject, description=description, exits=exits)

In [137]:
def parse_class_file(class_code):
    game_info = build_basic_game_info()

    raw_json = load_json(("%s.json" % class_code))
    subject = raw_json[class_code]['subject']

    rooms = build_plaza(subject=subject, code=class_code, children=raw_json[class_code]['children'].items())

    for child_key, section in raw_json[class_code]['children'].items():
        section_code = '%s_%s' % (class_code, child_key)
        
        rooms.update(build_building(subject=section['subject'], code=section_code, children=section['children'].items()))

        for key, value in section['children'].items():
            room_code = '%s_%s' % (section_code, key)
            room_subject = value['subject']
            exits = {
                'exit' : {
                    'displayName' : 'Lobby',
                    'destination' : section_code
                }
            }
            room = build_room(key=room_code, name=room_subject, description=room_subject, exits=exits)
            rooms.update(room)
    
    first_room = list(rooms.keys())[0]
    game_info['map'] = rooms
    game_info['player']['currentLocation'] = first_room
    write_json(game_info)

parse_class_file('A')